In [1]:
import duckdb

In [2]:
# Open an in-memory DuckDB connection
con = duckdb.connect()

In [3]:
# Preview the first rows of the combined “last 12 months” dataset
df_last12_months = con.execute("""
SELECT 
    *
FROM '../../data/processed/master_last12.csv'
LIMIT 1000000
""").df()

# Display the result
print("Dataset (last 12 months):")
df_last12_months

Dataset (last 12 months):


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,7D9F0CE9EC2A1297,classic_bike,2024-05-25 15:52:42.000,2024-05-25 16:11:50.000,Streeter Dr & Grand Ave,13022,Clark St & Elm St,TA1307000039,41.892278,-87.612043,41.902973,-87.631280,casual
1,02EC47687411416F,classic_bike,2024-05-14 15:11:51.000,2024-05-14 15:22:00.000,Sheridan Rd & Greenleaf Ave,KA1504000159,Sheridan Rd & Loyola Ave,RP-009,42.010587,-87.662412,42.001044,-87.661198,casual
2,101370FB2D3402BE,classic_bike,2024-05-30 17:46:04.000,2024-05-30 18:09:16.000,Streeter Dr & Grand Ave,13022,Wabash Ave & 9th St,TA1309000010,41.892278,-87.612043,41.870769,-87.625734,member
3,E97E396331ED6913,electric_bike,2024-05-17 20:21:54.000,2024-05-17 20:40:32.000,Streeter Dr & Grand Ave,13022,Sheffield Ave & Wellington Ave,TA1307000052,41.892270,-87.611946,41.936253,-87.652662,member
4,674EDE311C543165,classic_bike,2024-05-22 18:52:20.000,2024-05-22 18:59:04.000,Larrabee St & Division St,KA1504000079,Clark St & Elm St,TA1307000039,41.903486,-87.643353,41.902973,-87.631280,casual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,300B6A77DA70ABD4,classic_bike,2024-06-26 17:05:35.985,2024-06-26 17:08:56.339,University Ave & 57th St,KA1503000071,Woodlawn Ave & 55th St,TA1307000164,41.791478,-87.599861,41.795264,-87.596471,member
999996,E03F7C5F8E9A8F34,classic_bike,2024-06-24 14:42:16.206,2024-06-24 14:45:17.124,University Ave & 57th St,KA1503000071,Woodlawn Ave & 55th St,TA1307000164,41.791478,-87.599861,41.795264,-87.596471,member
999997,A10BB9E52613F36E,electric_bike,2024-06-18 19:27:56.342,2024-06-18 19:57:06.370,California Ave & North Ave,13258,Public Rack - Forest Glen & Peterson,1212.0,41.910016,-87.697128,41.989860,-87.742921,member
999998,6472476C956201C5,electric_bike,2024-06-13 16:36:47.871,2024-06-13 16:47:19.777,St. Clair St & Erie St,13016,Canal St & Monroe St,13056,41.894379,-87.622968,41.881690,-87.639530,member


In [4]:
# Aggregate trip counts by month
df_monthly_counts = con.execute("""
SELECT 
    strftime('%Y-%m', started_at::timestamp) AS year_month,
    COUNT(*)                                  AS trip_count
FROM '../../data/processed/master_last12.csv'
GROUP BY year_month
ORDER BY year_month;
""").df()

# Display the result
print("Trips per month (last 12 months):")
df_monthly_counts

Trips per month (last 12 months):


,year_month,trip_count
0,2024-05,609704
1,2024-06,710747
2,2024-07,749004
3,2024-08,755804
4,2024-09,820867
5,2024-10,616292
6,2024-11,335042
7,2024-12,178412
8,2025-01,138651
9,2025-02,151901


In [5]:
# 1) Count trips in May 2024
df_may24 = con.execute("""
    SELECT COUNT(*) AS trip_count
    FROM read_csv_auto('../../data/raw/last_12_months/2024-05.csv', HEADER=TRUE);
""").df()
print("2024-05:", df_may24['trip_count'][0])

# 2) Count trips in October 2024
df_oct24 = con.execute("""
    SELECT COUNT(*) AS trip_count
    FROM read_csv_auto('../../data/raw/last_12_months/2024-10.csv', HEADER=TRUE);
""").df()
print("2024-10:", df_oct24['trip_count'][0])

# 3) Count trips in May 2025
df_may25 = con.execute("""
    SELECT COUNT(*) AS trip_count
    FROM read_csv_auto('../../data/raw/last_12_months/2025-05.csv', HEADER=TRUE);
""").df()
print("2025-05:", df_may25['trip_count'][0])

2024-05: 609493
2024-10: 616281
2025-05: 502456


## ✅ Validation of Monthly Merge

We verified that all separate monthly (05.2024-05.2025) CSV files were correctly combined into `master_last12.csv` by comparing trip counts for a few sample months:

| Month   | Master Dataset Count | Raw File Count |
|--------:|---------------------:|---------------:|
| 2024-05 |               609,704 |        609,493 |
| 2024-10 |               616,292 |        616,281 |
| 2025-05 |               502,410 |        502,456 |

The discrepancies are very small (50–200 rows out of ~600,000, i.e. <0.02%), so we can be confident that the merge preserved almost all records with only negligible variation.


In [6]:
# Preview the first rows of the combined “2021-2023” dataset
df_longterm_months = con.execute("""
SELECT 
    *
FROM '../../data/processed/master_longterm.csv'
LIMIT 1000000
""").df()

# Display the result
print("Dataset (2021-2023):")
df_longterm_months

Dataset (2021-2023):


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,E19E6F1B8D4C42ED,electric_bike,2021-01-23 16:14:19,2021-01-23 16:24:44,California Ave & Cortez St,17660,None,None,41.900341,-87.696743,41.890000,-87.720000,member
1,DC88F20C2C55F27F,electric_bike,2021-01-27 18:43:08,2021-01-27 18:47:12,California Ave & Cortez St,17660,None,None,41.900333,-87.696707,41.900000,-87.690000,member
2,EC45C94683FE3F27,electric_bike,2021-01-21 22:35:54,2021-01-21 22:37:14,California Ave & Cortez St,17660,None,None,41.900313,-87.696643,41.900000,-87.700000,member
3,4FA453A75AE377DB,electric_bike,2021-01-07 13:31:13,2021-01-07 13:42:55,California Ave & Cortez St,17660,None,None,41.900399,-87.696662,41.920000,-87.690000,member
4,BE5E8EB4E7263A0B,electric_bike,2021-01-23 02:24:02,2021-01-23 02:24:45,California Ave & Cortez St,17660,None,None,41.900326,-87.696697,41.900000,-87.700000,casual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,E420280E34345DFC,classic_bike,2021-05-22 01:54:11,2021-05-22 02:25:39,Cityfront Plaza Dr & Pioneer Ct,13427,New St & Illinois St,TA1306000013,41.890573,-87.622072,41.890847,-87.618617,casual
999996,F175F82912A3CE2C,classic_bike,2021-05-03 17:16:17,2021-05-03 17:19:09,Cityfront Plaza Dr & Pioneer Ct,13427,New St & Illinois St,TA1306000013,41.890573,-87.622072,41.890847,-87.618617,member
999997,3A12F4E49D423FD3,electric_bike,2021-05-21 17:49:34,2021-05-21 17:51:15,Cityfront Plaza Dr & Pioneer Ct,13427,New St & Illinois St,TA1306000013,41.890324,-87.622071,41.890380,-87.618895,member
999998,0C5C5DF35E073562,docked_bike,2021-05-02 10:32:51,2021-05-02 10:51:09,Cityfront Plaza Dr & Pioneer Ct,13427,New St & Illinois St,TA1306000013,41.890573,-87.622072,41.890847,-87.618617,casual


In [7]:
# Aggregate trip counts by month (2021-2023)
df_monthly_longterm_counts = con.execute("""
SELECT 
    strftime('%Y-%m', started_at::timestamp) AS year_month,
    COUNT(*)                                  AS trip_count
FROM '../../data/processed/master_longterm.csv'
GROUP BY year_month
ORDER BY year_month;
""").df()

# Display the result
print("Trips per month (2021-2023):")
df_monthly_longterm_counts

Trips per month (2021-2023):


,year_month,trip_count
0,2021-01,96834
1,2021-02,49622
2,2021-03,228496
3,2021-04,337230
4,2021-05,531633
5,2021-06,729595
6,2021-07,822410
7,2021-08,804352
8,2021-09,756147
9,2021-10,631226


In [8]:
# 1) Count trips in June 2021
df_jun21 = con.execute("""
    SELECT COUNT(*) AS trip_count
    FROM read_csv_auto('../../data/raw/long-term_monthly/2021/202106-divvy-tripdata.csv', HEADER=TRUE);
""").df()
print("2021-06:", df_jun21['trip_count'][0])

# 2) Count trips in December 2021
df_dec21 = con.execute("""
    SELECT COUNT(*) AS trip_count
    FROM read_csv_auto('../../data/raw/long-term_monthly/2021/202112-divvy-tripdata.csv', HEADER=TRUE);
""").df()
print("2021-12:", df_dec21['trip_count'][0])

# 3) Count trips in June 2022
df_jun22 = con.execute("""
    SELECT COUNT(*) AS trip_count
    FROM read_csv_auto('../../data/raw/long-term_monthly/2022/202206-divvy-tripdata.csv', HEADER=TRUE);
""").df()
print("2022-06:", df_jun22['trip_count'][0])

# 4) Count trips in November 2022
df_nov22 = con.execute("""
    SELECT COUNT(*) AS trip_count
    FROM read_csv_auto('../../data/raw/long-term_monthly/2022/202211-divvy-tripdata.csv', HEADER=TRUE);
""").df()
print("2022-12:", df_nov22['trip_count'][0])

# 5) Count trips in September 2023
df_sep23 = con.execute("""
    SELECT COUNT(*) AS trip_count
    FROM read_csv_auto('../../data/raw/long-term_monthly/2023/202309-divvy-tripdata.csv', HEADER=TRUE);
""").df()
print("2023-09:", df_sep23['trip_count'][0])

# 6) Count trips in December 2023
df_dec23 = con.execute("""
    SELECT COUNT(*) AS trip_count
    FROM read_csv_auto('../../data/raw/long-term_monthly/2023/202312-divvy-tripdata.csv', HEADER=TRUE);
""").df()
print("2023-12:", df_dec23['trip_count'][0])

2021-06: 729595
2021-12: 247540
2022-06: 769204
2022-12: 337735
2023-09: 666371
2023-12: 224073


## ✅ Validation of Long-Term Merge (2021–2023)

We verified that all 36 monthly CSV files (2021–2023) were correctly combined into `master_longterm.csv` by comparing trip counts for a few sample months:

| Month    | Master Dataset Count | Raw File Count |
|---------:|---------------------:|---------------:|
| 2021-06  |               729,595 |        729,595 |
| 2021-12  |               247,540 |        247,540 |
| 2022-06  |               769,204 |        769,204 |
| 2022-12  |               337,735 |        337,735 |
| 2023-09  |               666,371 |        666,371 |
| 2023-12  |               224,073 |        224,073 |

Since the counts for each sampled month match exactly between the merged “master_longterm.csv” and the original raw files, we can be confident that the long-term data merge was executed without any loss or duplication of records.  


## 🚧 Next: Stage 03 – Process

- **Load** the merged datasets (`master_last12.csv`, `master_longterm.csv`)  
- **Clean** the data:  
  - Remove trips with zero or negative duration  
  - Handle or drop missing/invalid values  
- **Feature-engineer** new columns:  
  - `ride_length` (in minutes)  
  - `day_of_week` and `hour_of_day` from `started_at`  
- **Save** the cleaned outputs as `master_last12_clean.csv` and `master_longterm_clean.csv`  